In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request
import time
import random
import os

#### **Note** :  Le code étant le même pour chaque maladie à part l'équation de recherche, le même code sera utilisé plusieurs fois en changeant juste l'équation de recherche

#### Scraping de la première page de résultat avec ajout d'un test pour assurer la bonne récupération des mots-clés

In [ ]:
# URL de recherche sur PubMed avec l'équation de recherche
query = "(sleep habits OR sleep patterns OR sleep quality) AND (Alzheimer's disease OR cognitive decline OR dementia)"  #Alzheimer
url = f"https://pubmed.ncbi.nlm.nih.gov/?term={query.replace(' ', '+')}"

# Ajout d'un User-Agent pour éviter le blocage
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Envoi de la requête
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    articles = []

    # Recherche des articles dans la page
    for result in soup.find_all("article", class_="full-docsum"):
        title_tag = result.find("a", class_="docsum-title")
        title = title_tag.text.strip() if title_tag else "Titre inconnu"
        link = "https://pubmed.ncbi.nlm.nih.gov" + title_tag["href"] if title_tag else "Lien non disponible"

        # Initialisation des valeurs par défaut
        publication_date = "Date inconnue"
        summary = "Résumé non disponible"
        keywords = "Mots-clés non disponibles"
        authors = "Auteurs non disponibles"
        affiliations = "Affiliations non disponibles"

        if title_tag and title_tag["href"]:
            article_url = link
            article_response = requests.get(article_url, headers=headers)

            if article_response.status_code == 200:
                article_soup = BeautifulSoup(article_response.text, "html.parser")

                # Extraction de l'abstract et des mots-clés dans la div "abstract"
                abstract_div = article_soup.find("div", class_="abstract")
                if abstract_div:
                    # Extraction du résumé
                    summary_tag = abstract_div.find("div", class_="abstract-content")
                    if summary_tag:
                        paragraphs = summary_tag.find_all("p")
                        summary = " ".join([p.text.strip() for p in paragraphs]) if paragraphs else summary_tag.text.strip()

                    # Extraction des mots-clés avec une approche plus flexible
                    for p in abstract_div.find_all("p"):
                        strong_tag = p.find("strong", class_="sub-title")
                        if strong_tag and "Keywords" in strong_tag.text:  # Vérifie si "Keywords" est présent dans le texte
                            keywords = p.text.replace("Keywords:", "").strip()
                            print(f" Mots-clés extraits: {keywords}")
                            break  # On arrête dès qu'on trouve les mots-clés


                # Extraction de la date de publication
                date_tag = article_soup.find("span", class_="cit") or article_soup.find("time", class_="history-date")
                if date_tag:
                    publication_date = date_tag.text.strip()

                # Extraction des auteurs
                authors_tag = article_soup.find("div", class_="authors-list")
                if authors_tag:
                    authors = ", ".join([a.text.strip() for a in authors_tag.find_all("a")])

                # Extraction des affiliations
                affiliations_tag = article_soup.find("div", class_="affiliations")
                if affiliations_tag:
                    affiliations = "; ".join([aff.text.strip() for aff in affiliations_tag.find_all("li")])

            
            time.sleep(2)   # Pause pour éviter le blocage

        articles.append([title, publication_date, link, summary, keywords, authors, affiliations])

    # Création du dataFrame
    df = pd.DataFrame(articles, columns=["Titre", "Date de publication", "Lien", "Résumé", "Mots-clés", "Auteurs", "Affiliations"])

    # Sauvegarde en CSV
    df.to_csv("pubmed_articles_scraped.csv", index=False, encoding="utf-8")
else:
    print(f"Erreur lors de la requête : {response.status_code}")
